In [66]:
import os
import pandas as pd


directory = r"/home/krishna/Desktop/code/wind_power_time_series/data/"
#input('Please enter the folder directory path :')  # Replace with the actual directory path
#year_repo = input('Please enter the year :')

# List all files in the directory
files = os.listdir(directory)

turbine1_df = pd.DataFrame()
turbine2_df = pd.DataFrame()
turbine3_df = pd.DataFrame()
turbine4_df = pd.DataFrame()
turbine5_df = pd.DataFrame()
turbine6_df = pd.DataFrame()

turbines = [turbine1_df, turbine2_df, turbine3_df,turbine4_df, turbine5_df, turbine6_df ]


csv_1_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_1')]
csv_2_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_2')]
csv_3_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_3')]
csv_4_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_4')]
csv_5_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_5')]
csv_6_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_6')]

csv_i_files  = [csv_1_files,csv_2_files,csv_3_files,csv_4_files,csv_5_files,csv_6_files]


In [67]:
turbine1_df = pd.DataFrame()
for csv_1_file in csv_1_files:
    path = r'/home/krishna/Desktop/code/wind_power_time_series/data/'
    df = pd.read_csv(f'{path}/{csv_1_file}',skiprows=8, header=1)
    print(df.size)
    turbine1_df = pd.concat([turbine1_df,df], axis=0)
    print(turbine1_df.size)
    

7793136
7793136
15758496
23551632
15715440
39267072


In [68]:
import data_preprocess as dpf
turbine1_df_final = dpf.data_process(turbine1_df)


In [69]:
turbine1_df_final = turbine1_df_final.dropna(subset=['Wind speed (m/s)'])

turbine1_df_final['Metal particle count counter'].fillna(method='ffill', inplace=True)
turbine1_df_final['Tower Acceleration y (mm/ss)'].fillna(method='ffill', inplace=True)
turbine1_df_final['Tower Acceleration X (mm/ss)'].fillna(method='ffill', inplace=True)
turbine1_df_final['blade_angle'].fillna(method='ffill', inplace=True)
turbine1_df_final['Front bearing temperature (°C)'].fillna(method='ffill', inplace=True)
turbine1_df_final['Rear bearing temperature (°C)'].fillna(method='ffill', inplace=True)
turbine1_df_final['Nacelle ambient temperature (°C)'].fillna(method='ffill', inplace=True)

for i in (1,50):
    means = turbine1_df_final.rolling(window=7, min_periods=1).mean(numeric_only=True)

    # replace NaN values in each column with the corresponding median value, or 0 if there are no non-NaN values
    turbine1_df_final = turbine1_df_final.fillna(means)





In [70]:
turbine1_df_final = turbine1_df_final.reset_index(drop = True)


In [71]:
import pandas as pd


# specify the list of features to lag
diff_features = ['blade_angle','Nacelle ambient temperature (°C)','Tower Acceleration X (mm/ss)','Tower Acceleration y (mm/ss)']
features_to_lag = ['Wind direction (°)','Rear bearing temperature (°C)',
                'Rotor speed (RPM)','Generator RPM (RPM)',
                'Front bearing temperature (°C)','Tower Acceleration X (mm/ss)','Wind speed (m/s)','Tower Acceleration y (mm/ss)','Metal particle count counter']
print("Start")
# lag the specified features by 1, 2, 3, 4, and 5 timesteps
for feature in features_to_lag:
    for i in range(1, 6):
        feature_n = f'{feature}_lagged_{i}'
        turbine1_df_final[feature_n] = turbine1_df_final[feature].shift(i)

for feature in diff_features:
    feature_n = f'{feature}_diff'
    turbine1_df_final[feature_n] = turbine1_df_final[feature].diff()
               

# fill missing values with the previous value
turbine1_df_final.fillna(method='ffill', inplace=True)


Start


In [72]:
turbine1_df_final.fillna(0, inplace=True)



In [73]:
print(turbine1_df_final.isnull().sum())

# Date and time                          0
Wind speed (m/s)                         0
Wind direction (°)                       0
Nacelle position (°)                     0
Power (kW)                               0
                                        ..
Metal particle count counter_lagged_5    0
blade_angle_diff                         0
Nacelle ambient temperature (°C)_diff    0
Tower Acceleration X (mm/ss)_diff        0
Tower Acceleration y (mm/ss)_diff        0
Length: 63, dtype: int64


In [74]:
turbine1_df_final_numeric = turbine1_df_final.select_dtypes(include='int')
print(turbine1_df_final.columns)


Index(['# Date and time', 'Wind speed (m/s)', 'Wind direction (°)',
       'Nacelle position (°)', 'Power (kW)', 'Front bearing temperature (°C)',
       'Rear bearing temperature (°C)', 'Nacelle ambient temperature (°C)',
       'Rotor speed (RPM)', 'Generator RPM (RPM)',
       'Metal particle count counter', 'Tower Acceleration X (mm/ss)',
       'Tower Acceleration y (mm/ss)', 'blade_angle',
       'Wind direction (°)_lagged_1', 'Wind direction (°)_lagged_2',
       'Wind direction (°)_lagged_3', 'Wind direction (°)_lagged_4',
       'Wind direction (°)_lagged_5', 'Rear bearing temperature (°C)_lagged_1',
       'Rear bearing temperature (°C)_lagged_2',
       'Rear bearing temperature (°C)_lagged_3',
       'Rear bearing temperature (°C)_lagged_4',
       'Rear bearing temperature (°C)_lagged_5', 'Rotor speed (RPM)_lagged_1',
       'Rotor speed (RPM)_lagged_2', 'Rotor speed (RPM)_lagged_3',
       'Rotor speed (RPM)_lagged_4', 'Rotor speed (RPM)_lagged_5',
       'Generator RPM (

In [ ]:
import pandas as pd
from datetime import datetime

# Assuming 'data_20_final' is your DataFrame containing the data
turbine1_df_final['# Date and time'] = pd.to_datetime(turbine1_df_final['# Date and time'])
turbine1_df_final['# Date and time'] = turbine1_df_final['# Date and time'].apply(lambda x: int(x.timestamp()))

# Now, you can proceed with the normalization step


In [ ]:
data_20_final_numeric = turbine1_df_final.select_dtypes(include='int')
turbine1_df_final = dpf.normalize_all(turbine1_df_final)


In [ ]:
turbine1_df_final.fillna(method='bfill', inplace=True)
turbine1_df_final.fillna(method='bfill', inplace=True)
turbine1_df_final.fillna(method='bfill', inplace=True)
turbine1_df_final.fillna(method='bfill', inplace=True)


In [ ]:
turbine1_df_final.to_csv('input_data.csv', index_label=False)


In [ ]:
import pickle
turbine1_df_final.to_pickle('turbine1_df_final.pkl')
